In [233]:
import base64
import bz2
import importlib
import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F

model_path = "models/latest.pth"
# model_path = "ds/models/offensive_4439.pth"

weights = torch.load(model_path)

PARAM = base64.b64encode(bz2.compress(pickle.dumps(weights)))
state_dict = pickle.loads(bz2.decompress(base64.b64decode(PARAM)))

In [238]:
%%writefile handyrl/envs/kaggle/geese/rule_based_goose.py
# This is a lightweight ML agent trained by self-play.
# After sharing this notebook,
# we will add Hungry Geese environment in our HandyRL library.
# https://github.com/DeNA/HandyRL
# We hope you enjoy reinforcement learning!


import base64
import bz2
import math
import pickle
from collections import defaultdict, deque

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

# Neural Network for Hungry Geese

class TorusConv2d(nn.Module):
    def __init__(self, input_dim, output_dim, kernel_size, bn, groups=1):
        super().__init__()
        self.edge_size = (kernel_size[0] // 2, kernel_size[1] // 2)
        self.conv = nn.Conv2d(input_dim, output_dim, kernel_size=kernel_size, groups=groups)
        self.bn = nn.BatchNorm2d(output_dim) if bn else None

    def forward(self, x):
        h = torch.cat([x[:, :, :, -self.edge_size[1]:], x, x[:, :, :, :self.edge_size[1]]], dim=3)
        h = torch.cat([h[:, :, -self.edge_size[0]:], h, h[:, :, :self.edge_size[0]]], dim=2)
        h = self.conv(h)
        h = self.bn(h) if self.bn is not None else h
        return h

class MLP(nn.Module):
    def __init__(self, embed_dim, dropout):
        super(MLP, self).__init__()
        self.c_fc = nn.Linear(embed_dim, embed_dim*4)
        self.c_proj = nn.Linear(embed_dim*4, embed_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        h = F.relu(self.c_fc(x))
        h2 = self.c_proj(h)
        return self.dropout(h2)

class Block(nn.Module):
    def __init__(self, embed_dim, num_heads, dropout):
        super(Block, self).__init__()
        self.attn = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads, dropout=dropout)
        self.ln_1 = nn.LayerNorm(embed_dim)
        self.mlp = MLP(embed_dim, dropout)
        self.ln_2 = nn.LayerNorm(embed_dim) 

    def forward(self, x):
        a, _ = self.attn(x, x, x) 
        n = self.ln_1(x + a)
        m = self.mlp(n)
        h = self.ln_2(n + m)
        return h

import copy
class GeeseNet(nn.Module):
    def __init__(self, spacial_input_dim=4*4+1, scalar_input_dim=1, embed_dim=64, n_layers=8, dropout=0.0):
        super().__init__()
        self.embed = nn.Linear(spacial_input_dim, embed_dim)
        self.blocks = nn.ModuleList([TorusConv2d(embed_dim, embed_dim, (3, 3), True) for _ in range(n_layers)])
        self.mlp = MLP(embed_dim+scalar_input_dim, dropout)

        self.head_p = nn.Linear(embed_dim+scalar_input_dim, 4, bias=False)
        self.head_v = nn.Linear(embed_dim+scalar_input_dim, 1, bias=False)

    def forward(self, x, _=None):
        x_spacial, x_scalar = x
        h = self.embed(x_spacial)
        h = h.permute(0, 3, 1, 2)
        for block in self.blocks:
            h = F.relu_(h + block(h))

        h = torch.squeeze(torch.squeeze(h[:, :, 0:1, 0:1], dim=-1), dim=-1)
        h = torch.cat([h, x_scalar], dim=-1)
        h = self.mlp(h)

        p = F.softmax(self.head_p(h), dim=-1)
        v = torch.tanh(self.head_v(h))

        return {"policy": p, "value": v}



# Input for Neural Network


NUM_ROW = 7
NUM_COL = 11
CENTER_ROW = NUM_ROW // 2
CENTER_COL = NUM_COL // 2


def to_offset(x):
    row = CENTER_ROW - x // NUM_COL
    col = CENTER_COL - x % NUM_COL
    return row, col


def to_row(offset, x):
    return (x // NUM_COL + offset) % NUM_ROW


def to_col(offset, x):
    return (x + offset) % NUM_COL


def distance(a, b):
    x = b[0] - a[0]
    y = b[1] - a[1]
    return (x, y), abs(x) + abs(y)


def around(a):
    return [
        ((a[0] - 1) % 7, a[1]),
        ((a[0] + 1) % 7, a[1]),
        (a[0], (a[1] - 1) % 11),
        (a[0], (a[1] + 1) % 11),
    ]


def empty_around_head(field, head, x):
    return [e for e in around(x) if field[e[0], e[1]] <= (distance(head, x)[1] + 1)]


def bfs_close_route(field, head):
    q = deque([head])
    searched = defaultdict(bool)
    max_dist = 0
    while len(q) != 0:
        v = q.popleft()
        searched[v] = True
        # 迂回した距離が計算できていない
        max_dist = max(max_dist, distance(head, v)[1])
        edges = [a for a in empty_around_head(field, head, v) if not searched[a]]
        for edge in edges:
            q.append(edge)
    return max_dist


def make_input(obses):
    b = np.zeros((17, 7 * 11), dtype=np.float32)
    obs = obses[-1]

    for p, pos_list in enumerate(obs["geese"]):
        # head position
        for pos in pos_list[:1]:
            b[0 + (p - obs["index"]) % 4, pos] = 1
        # tip position
        for pos in pos_list[-1:]:
            b[4 + (p - obs["index"]) % 4, pos] = 1
        # whole position
        for pos in pos_list:
            b[8 + (p - obs["index"]) % 4, pos] = 1

    # previous head position
    if len(obses) > 1:
        obs_prev = obses[-2]
        for p, pos_list in enumerate(obs_prev["geese"]):
            for pos in pos_list[:1]:
                b[12 + (p - obs["index"]) % 4, pos] = 1

    # food
    for pos in obs["food"]:
        b[16, pos] = 1

    return b.reshape(-1, 7, 11)


def make_input_centering_head(obses):
    b = np.zeros((17, 7, 11), dtype=np.float32)
    obs = obses[-1]

    player_goose_head = obs["geese"][obs["index"]][0]
    o_row, o_col = to_offset(player_goose_head)

    for p, geese in enumerate(obs["geese"]):
        # whole position
        for pos in geese:
            b[0 + (p - obs["index"]) % 4, to_row(o_row, pos), to_col(o_col, pos)] = 1
        # tip position
        for pos in geese[-1:]:
            b[4 + (p - obs["index"]) % 4, to_row(o_row, pos), to_col(o_col, pos)] = 1
        # head position
        for pos in geese[:1]:
            b[8 + (p - obs["index"]) % 4, to_row(o_row, pos), to_col(o_col, pos)] = 1

    # previous head position
    if len(obses) > 1:
        obs_prev = obses[-2]
        for p, geese in enumerate(obs_prev["geese"]):
            for pos in geese[:1]:
                b[12 + (p - obs["index"]) % 4, to_row(o_row, pos), to_col(o_col, pos)] = 1

    # food
    for pos in obs["food"]:
        b[16, to_row(o_row, pos), to_col(o_col, pos)] = 1

    return b


def make_input_2step(obses):
    b = np.zeros((25, 7 * 11), dtype=np.float32)

    obs = obses[-1]
    for p, geese in enumerate(obs["geese"]):
        # head position
        for pos in geese[:1]:
            b[0 + (p - obs["index"]) % 4, pos] = 1
        # tip position
        for pos in geese[-1:]:
            b[4 + (p - obs["index"]) % 4, pos] = 1
        # whole position
        for pos in geese:
            b[8 + (p - obs["index"]) % 4, pos] = 1

    if len(obses) > 1:
        obs = obses[-2]
        for p, geese in enumerate(obs["geese"]):
            # head position
            for pos in geese[:1]:
                b[12 + (p - obs["index"]) % 4, pos] = 1
            # tip position
            for pos in geese[-1:]:
                b[16 + (p - obs["index"]) % 4, pos] = 1
            # whole position
            for pos in geese:
                b[20 + (p - obs["index"]) % 4, pos] = 1

    # food
    for pos in obs["food"]:
        b[24, pos] = 1

    return b.reshape(-1, 7, 11)


def make_input_centering_head_for_rule(obses):
    b = {}
    for i in range(4):
        b[i] = defaultdict(list)
    obs = obses[-1]

    player_goose_head = obs["geese"][obs["index"]][0]
    o_row, o_col = to_offset(player_goose_head)

    for p, geese in enumerate(obs["geese"]):
        # whole position
        for pos in geese:
            b[(p - obs["index"]) % 4]["body"].append((to_row(o_row, pos), to_col(o_col, pos)))

    # previous head position
    if len(obses) > 1:
        obs_prev = obses[-2]
        for p, geese in enumerate(obs_prev["geese"]):
            for pos in geese[:1]:
                b[(p - obs["index"]) % 4]["previous"].append((to_row(o_row, pos), to_col(o_col, pos)))

    # food
    for pos in obs["food"]:
        b[0]["food"].append((to_row(o_row, pos), to_col(o_col, pos)))

    return b


def make_input_num_turn_of_fill(obses):
    b = np.zeros((7, 11), dtype=np.float32)
    obs = obses[-1]

    player_goose_head = obs["geese"][obs["index"]][0]
    o_row, o_col = to_offset(player_goose_head)

    for p, geese in enumerate(obs["geese"]):
        # マスが、何ターン後に空くか
        for i, pos in enumerate(geese[::-1]):
            b[to_row(o_row, pos), to_col(o_col, pos)] = i + 1

        # 自分の頭は、逆に進めないので、空くのは1周後
        if (p - obs["index"]) % 4 == 0:
            for pos in geese[:1]:
                if b[to_row(o_row, pos), to_col(o_col, pos)] < 4:
                    b[to_row(o_row, pos), to_col(o_col, pos)] = 4

    # previous head position
    # 自分が直前にいた場所も、最短で、一周後
    if len(obses) > 1:
        obs_prev = obses[-2]
        for p, geese in enumerate(obs_prev["geese"]):
            if (p - obs["index"]) % 4 == 0:
                for pos in geese[:1]:
                    if b[to_row(o_row, pos), to_col(o_col, pos)] < 3:
                        b[to_row(o_row, pos), to_col(o_col, pos)] = 3

    return b

def make_input_shunpi(obses):

    x_spacial = np.zeros((7, 11, 4*4 + 1), dtype=np.float32)
    x_scalar = np.zeros(1, dtype=np.float32)
    obs = obses[-1]
    player = obs["index"]

    player_goose_head = obs['geese'][player][0]
    o_row = (7 - player_goose_head // 11) % 7
    o_col = (11 - player_goose_head % 11) % 11

    for p, geese in enumerate(obs['geese']):
        pid = (p - player + 4) % 4
        # head position
        for pos in geese[:1]:
            x_spacial[to_row(o_row, pos), to_col(o_col, pos), 4*0 + pid] = 1
        # tip position
        for pos in geese[-1:]:
            x_spacial[to_row(o_row, pos), to_col(o_col, pos), 4*1 + pid] = 1
        # whole position(head寄り、tail寄り)
        for i in range(len(geese)):
            if i == 0 or i == len(geese) - 1:
                continue
            x_spacial[to_row(o_row, pos), to_col(o_col, pos), 4*2 + pid] = (len(geese) - 1 - i) / len(geese)
            x_spacial[to_row(o_row, pos), to_col(o_col, pos), 4*3 + pid] = i / len(geese)

    # x座標(0~6)
    #for pos in range(77):
    #    x_spacial[to_row(o_row, pos), to_col(o_col, pos), 4*4 + to_row(o_row, pos)] = 1

    # y座標(0~10)
    #for pos in range(77):
    #    x_spacial[to_row(o_row, pos), to_col(o_col, pos), 4*4 + 7 + to_col(o_row, pos)] = 1

    # food
    for pos in obs['food']:
        x_spacial[to_row(o_row, pos), to_col(o_col, pos), 4*4] = 1

    #(step数 - 100) / 100
    x_scalar[0] = (len(obses) - 100) / 100

    #return x_spacial.reshape(7*11, -1), x_scalar
    return x_spacial, x_scalar

def apply_rule(b, prob):
    """
    player head = (3, 5)
    ["NORTH", "SOUTH", "WEST", "EAST"]
    """
    north = (2, 5)
    south = (4, 5)
    west = (3, 4)
    east = (3, 6)
    neighbor = [north, south, west, east]

    # 隣接している場所に行けないケース
    for i, n in enumerate(neighbor):
        # 自分の直前の場所
        if n in b[0]["previous"]:
            prob[i] = -np.inf

        for p in range(4):
            # ガチョウの体がある場所 (しっぽ除く)
            if n in b[p]["body"][:-1]:
                prob[i] = -np.inf

    # 次の移動で頭がぶつかる可能性のあるケース
    for p in range(1, 4):
        if b[p]["body"] != []:
            (x, y), d = distance(b[0]["body"][0], b[p]["body"][0])
            if d == 2:
                if x < 0:
                    prob[0] -= 100 if north in b[0]["food"] else 50
                elif x > 0:
                    prob[1] -= 100 if south in b[0]["food"] else 50
                if y < 0:
                    prob[2] -= 100 if west in b[0]["food"] else 50
                elif y > 0:
                    prob[3] -= 100 if east in b[0]["food"] else 50

    # しっぽが伸びる可能性のあるケース
    for i, n in enumerate(neighbor):
        for p in range(1, 4):
            if (
                b[p]["body"] != []
                and n == b[p]["body"][-1]
                and any(food in around(b[p]["body"][0]) for food in b[0]["food"])
            ):
                prob[i] -= 100

    return prob


def apply_rule2(b, prob):
    """
    player head = (3, 5)
    ["NORTH", "SOUTH", "WEST", "EAST"]
    """
    north = (2, 5)
    south = (4, 5)
    west = (3, 4)
    east = (3, 6)
    neighbor = [north, south, west, east]

    # 閉域探索
    for i, n in enumerate(neighbor):
        if bfs_close_route(b, n) <= 3:
            prob[i] -= 10_000

    return prob


# Load PyTorch Model


PARAM = b"xxxxxxxxxx"

state_dict = pickle.loads(bz2.decompress(base64.b64decode(PARAM)))
model = GeeseNet()
# model = GeeseNetA()
model.load_state_dict(state_dict)
model.eval()


# Main Function of Agent

obses = []


def agent(obs, _):
    obses.append(obs)

    x = make_input_shunpi(obses)
    # x = make_input_2step(obses)
    # x = make_input_centering_head(obses)
    y = make_input_centering_head_for_rule(obses)
    # z = make_input_num_turn_of_fill(obses)

    with torch.no_grad():
        x1, x2 = x
        xt = torch.from_numpy(x1).unsqueeze(0), torch.from_numpy(x2).unsqueeze(0)
        o = model(xt)
    p = o["policy"].squeeze(0).detach().numpy()

    p = apply_rule(y, p)
    # p = apply_rule2(z, p)

    print(f"player: {obs['index']}, actions: {p}")

    actions = ["NORTH", "SOUTH", "WEST", "EAST"]
    return actions[np.argmax(p)]

Overwriting handyrl/envs/kaggle/geese/rule_based_goose.py


In [239]:
# Read in the submission file
goose_file = "handyrl/envs/kaggle/geese/rule_based_goose.py"

with open(
    goose_file, encoding="utf-8"
) as file:
    filedata = file.read()

# Replace the target string
filedata = filedata.replace("xxxxxxxxxx", PARAM.decode("utf-8"))

# Write the file out again
with open(goose_file, "w") as file:
    file.write(filedata)

In [240]:
from kaggle_environments import make

env = make("hungry_geese", debug=True)
env.reset()
env.run([goose_file, goose_file, goose_file, goose_file])
env.render(mode="ipython", width=500, height=400)

# env_module = importlib.import_module("handyrl.envs.kaggle.hungry_geese")
# e = env_module.Environment()
# e.reset()
#
# while not e.terminal()
#     obs = e.observation()
#     actions = {}
#     for player in e.turns():
#         actions[player] = e.rule_based_action_smart_geese(player, goose="rule_based_goose")
#     e.step(actions)
#     rewards = e.reward()
#     print(rewards)
# e.outcome()

player: 0, actions: [0.3272816  0.23833679 0.22345266 0.21092898]
player: 1, actions: [0.330288   0.2374358  0.22327673 0.20899947]
player: 2, actions: [0.32055336 0.242615   0.22255506 0.21427661]
player: 3, actions: [0.31771198 0.24169964 0.22510114 0.21548717]
player: 0, actions: [0.32908502       -inf 0.2230193  0.2110513 ]
player: 1, actions: [0.32715186       -inf 0.2258105  0.20972455]
player: 2, actions: [0.32025686       -inf 0.22287492 0.214317  ]
player: 3, actions: [0.31551737       -inf 0.22814512 0.21713652]
player: 0, actions: [0.32512388       -inf 0.22306825 0.21103597]
player: 1, actions: [0.32230547       -inf 0.2270567  0.20942797]
player: 2, actions: [0.32047486       -inf 0.22275142 0.21493776]
player: 3, actions: [0.30728763       -inf 0.23677441 0.21656945]
player: 0, actions: [0.32475907       -inf 0.22285461 0.21362214]
player: 1, actions: [0.32567865       -inf 0.2239319  0.21201535]
player: 2, actions: [0.3199785        -inf 0.22268136 0.21505377]
player: 3,

In [124]:
# 0: 白
# 1: 青
# 2: 緑
# 3: 赤